<a href="https://colab.research.google.com/github/n0k0m3/rnn-mal-sentiment/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install -U tables
!apt-get update && apt-get install git-lfs
!git lfs clone https://github.com/n0k0m3/rnn-mal-sentiment

WARNING: Restart Runtime after running the 1st cell

In [1]:
%cd rnn-mal-sentiment
import pandas as pd
import numpy as np
import tensorflow as tf

/content/rnn-mal-sentiment


In [2]:
df = pd.read_hdf("data.h5")

In [3]:
df

,overall,story,animation,sound,character,enjoyment,review
0,0,0,0,0,0,0,test review
1,10,10,10,10,9,10,Death Note is definitely one of the best anime...
2,8,8,9,9,8,9,Death Note. Another famous anime. But this tim...
3,10,10,10,10,10,10,"Allow me to explain my love of ""the pay off"" i..."
4,7,7,6,8,7,7,May contain light spoilers...\r\n\r\nDeath Not...
...,...,...,...,...,...,...,...
84859,8,7,10,9,8,5,SPOILEROONI'S (that means spoiler's)\r\n\r\nNa...
84860,9,8,9,8,7,9,"Such a fun movie. It's not perfect, but it get..."
84861,8,7,9,8,9,8,This movie does a lot of things right. It intr...
84862,8,7,8,8,8,8,"It starts out strong, the first half is excell..."
